# GUI - ENDING

This is the ending GUI of the project

This GUI will show the complete result analysis of each feature and the status (Pass/ Fail) of each feature

In [ ]:
from tkinter import *
from PIL import Image as PIL_Image, ImageTk
import cv2
from tkinter import messagebox

# Retrieving the data stored by the previously running notebooks 
# (Ensure to run %store -r result_list and %store -r path in previous notebooks)

# Result list: contains complete result analysis of each feature
%store -r result_list         

# Path of input image
%store -r path                


# Function to display the output
def display_output():
    global path
    
    # Creating the root window
    root = Tk()
    root.title('Fake Currency Detection - Result Analysis')
    
    # Defining attributes of root window
    root.resizable(False, False)  # This code helps to disable windows from resizing

    window_width = 1100
    window_height = 600

    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()

    x_cordinate = int((screen_width/2) - (window_width/2))
    y_cordinate = int((screen_height/2) - (window_height/2))

    root.geometry("{}x{}+{}+{}".format(window_width, window_height, x_cordinate, y_cordinate))

    # Creating a main frame inside the root window
    main_frame = Frame(root, relief=GROOVE, bd=1)
    main_frame.place(x=10, y=10)  # Placing the frame at (10, 10)

    # Creating a canvas inside main_frame
    canvas = Canvas(main_frame)
    master_frame = Frame(canvas)  # Creating master_frame inside the canvas

    # Inserting and configuring scrollbar widget
    myscrollbar = Scrollbar(main_frame, orient="vertical", command=canvas.yview)
    canvas.configure(yscrollcommand=myscrollbar.set)

    myscrollbar.pack(side="right", fill="y")
    canvas.pack(side="left")
    canvas.create_window((0, 0), window=master_frame, anchor='nw')

    def on_configure(event):
        # Update scrollregion when content size changes
        canvas.configure(scrollregion=canvas.bbox('all'))

    canvas.bind('<Configure>', on_configure)

    # Displaying the analysis results
    pass_count = 0

    for i, feature_data in enumerate(result_list):
        feature_num = i + 1
        
        # Creating a frame for each feature's display
        feature_frame = Frame(master_frame, relief=RAISED, borderwidth=1, bg='light blue')
        feature_frame.grid(row=i, column=0, padx=20, pady=20, sticky="nsew")

        # Feature number label
        label1 = Label(feature_frame, text=f"Feature {feature_num}", fg='black', font="Verdana 12 bold")
        label1.pack()

        # Displaying the image
        image = feature_data[0].copy()
        h, w = image.shape[:2]
        aspect_ratio = w / h

        if h > w:
            resize_height = 200
            resize_width = aspect_ratio * resize_height
            img_x = (200 - resize_width) / 2
            img_y = 0
        elif h < w:
            resize_width = 200
            resize_height = resize_width / aspect_ratio
            img_x = 0
            img_y = (200 - resize_height) / 2
        else:
            resize_height = 200
            resize_width = 200
            img_x = 0
            img_y = 0

        resize_height = int(resize_height)
        resize_width = int(resize_width)
        img_x = int(img_x)
        img_y = int(img_y)

        image = cv2.resize(image, (resize_width, resize_height))
        image = PIL_Image.fromarray(image)
        image = ImageTk.PhotoImage(image)

        canvas_image = Canvas(feature_frame, width=200, height=200, bg='white')
        canvas_image.pack()

        canvas_image.create_image(img_x, img_y, anchor=NW, image=image)
        canvas_image.image = image

        # Average SSIM Score label
        avg_score = feature_data[1]
        avg_score_label = Label(feature_frame, text=f"Avg. SSIM Score: {avg_score:.3f}", fg='dark blue',
                                font="Verdana 11", bg='light blue')
        avg_score_label.pack()

        # Max SSIM Score label (if applicable)
        if len(feature_data) > 2:
            max_score = feature_data[2]
            max_score_label = Label(feature_frame, text=f"Max. SSIM Score: {max_score:.3f}", fg='dark blue',
                                    font="Verdana 11", bg='light blue')
            max_score_label.pack()

        # Status label (PASS/FAIL)
        status = feature_data[-1]
        if status:
            pass_count += 1
            status_label = Label(feature_frame, text="Status: PASS!", fg='green', font="Verdana 11 bold",
                                 bg='light blue')
            status_label.pack()
        else:
            status_label = Label(feature_frame, text="Status: FAIL!", fg='red', font="Verdana 11 bold",
                                 bg='light blue')
            status_label.pack()

    # Result label
    result_label = Label(master_frame, text=f"RESULT: {pass_count} / {len(result_list)} features PASSED!", fg='green',
                         font="Verdana 24 bold")
    result_label.grid(row=len(result_list), column=0, pady=10)

    canvas.bind_all("<MouseWheel>", lambda event: canvas.yview_scroll(int(-1 * (event.delta / 120)), "units"))

    root.mainloop()


# Call the display_output function to start the GUI
display_output()
